# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [8]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [3]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [27]:
# Import the CSVs to Pandas DataFrames
file_path = Path("toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

In [31]:
df_neighbourhood_locations.head()

,neighbourhood,lat,lon
0,Agincourt North,43.805441,-79.266712
1,Agincourt South-Malvern West,43.788658,-79.265612
2,Alderwood,43.604937,-79.541611
3,Annex,43.671585,-79.404001
4,Banbury-Don Mills,43.737657,-79.349718


- - -

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [6]:
# Getting the data from the top 10 expensive neighbourhoods
def top_most_expensive_neighbourhoods():
    df_top_exp = to_data.groupby('neighbourhood').sum()
    df_top_avg= df_top_exp['average_house_value'].sort_values(ascending=False).nlargest(10)
    fig_top_avg = plt.figure()
    plot_top_avg = df_top_avg.plot.bar()
    plot_top_avg.set_xlabel("neighbourhood")
    plot_top_avg.set_ylabel("average_house_value")
    plot_top_avg.set_title("The top 10 expensive neighbourhoods")
    plt.close(fig_top_exp)
    return pn.pane.Matplotlib(fig_top_avg, tight=True)
    
# Calculate the mean number of dwelling types units per year
# YOUR CODE HERE!
def number_of_dwelling_types_units_per_year():
    df_num_dwelling = to_data['movable_dwelling'].groupby(to_data.index).mean()
    fig_num_dwelling = plt.figure()
    plot_num_dwelling = df_num_dwelling.plot.bar()
    plot_num_dwelling.set_xlabel("Year")
    plot_num_dwelling.set_ylabel("Mean number of dwelling types")
    plot_num_dwelling.set_title("The mean number of dwelling types units per year")
    plt.close(fig_num_dwelling)
    return pn.pane.Matplotlib(fig_num_dwelling, tight=True)
# Calculate the average monthly shelter costs for owned and rented dwellings
def average_monthly_shelter_cost():
    df_monthly_shelter_cost = to_data.groupby(to_data.index).mean()
    df_monthly_shelter_cost= df_monthly_shelter_cost.drop(['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house','average_house_value'], axis =1)    
    fig_monthly_shelter_cost = plt.figure()
    plot_monthly_shelter_cost = df_monthly_shelter_cost.plot.bar()
    plot_monthly_shelter_cost.set_xlabel("shelter_costs_owned", "shelter_costs_rented")
    plot_monthly_shelter_cost.set_ylabel("Mean number of shelter_cost")
    plot_monthly_shelter_cost.set_title("The mean number of shelter cost for owed and rented dwelling")
    plt.close(fig_monthly_shelter_cost)
    return pn.pane.Matplotlib(fig_monthly_shelter_cost, tight=True)


### Panel Visualization Functions

In [32]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
   
    # YOUR CODE HERE!
    neighbourhood_map= to_data.groupby("neighbourhood").mean()
    neighbourhood_map = neighbourhood_map.reset_index()
    neighbourhood_map = pd.merge(df_neighbourhood_locations,neighbourhood_map, on = "neighbourhood")
    px.set_mapbox_access_token(map_box_api)
    px.set_mapbox_access_token("pk.eyJ1IjoiYW1icmVlbnNpbW9uMjAwMiIsImEiOiJja2dpcGlrdXgwamJ2Mnpuc2Q3c2tqMzd6In0.Ey8qXfQ7cvOasEUdcQiedQ")
    map = px.scatter_mapbox(
    neighbourhood_map,
    lat="lat",
    lon="lon",
    color="average_house_value",
    size="average_house_value",
    size_max=15,
    zoom=10)
    plotly_panel = pn.pane.Plotly(map)
    plotly_panel._updates = True
    return plotly_panel
def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    fig = plt.figure()
    bar = data.plot.bar(color=color)
    Set_xlabel= xlabel 
    set_ylabel= ylabel
    title= title 
    plt.show()
    plt.close(fig)

    # YOUR CODE HERE!

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    fig = plt.figure()
    line = data.plot.line(color= color)
    Set_xlabel= xlabel 
    set_ylabel= ylabel
    title= title 
    plt.show()
    plt.close(fig)
    # YOUR CODE HERE!

def average_house_value():
    """Average house values per year."""
    
    # YOUR CODE HERE!
    average_house_value = to_data[["average_house_value"]].groupby("year","neighbourhood").mean()
    
    return average_house_value
def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    # YOUR CODE HERE!
    df_data_frame = to_data
    df_data_frame = df_data_frame.drop(['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house','duplex','apartment_five_storeys_less','other_house','shelter_costs_owned','shelter_costs_rented'], axis =1)   
    df_avg_house = df_data_frame.hvplot.line("year","average_house_value", groupby = "neighbourhood")
    return df_avg_house
def number_dwelling_types():
    """Number of dwelling types per year"""
    
    # YOUR CODE HERE!
    df_num_dwelling = to_data['movable_dwelling'].groupby(to_data.index).mean()
    fig_num_dwelling = plt.figure()
    plot_num_dwelling = df_num_dwelling.plot.bar()
    plot_num_dwelling.set_xlabel("Year")
    plot_num_dwelling.set_ylabel("Mean number of dwelling types")
    plot_num_dwelling.set_title("The mean number of dwelling types units per year")
    plt.close(fig_num_dwelling)
    return pn.pane.Matplotlib(fig_num_dwelling, tight=True)

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    average_house_value_snapshot = to_data[["average_house_value"]].groupby("year","neighbourhood").mean()
    fig_average_house_value_snapshot= plt.figure()
    plot_average_house_value_snapshot = df_average_house_value_snapshot.plot.line()
    plot_average_house_value_snapshot.set_xlabel("year")
    plot_average_house_value_snapshot.set_ylabel("avg house value")
    plot_average_house_value_snapshot.set_title("average_house_value")
    plt.close(fig_average_house_value_snapshot)
    return pn.pane.Matplotlib(fig_average_house_value_snapshot, tight=True)
    # YOUR CODE HERE!

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    # YOUR CODE HERE!
    df_top_exp = to_data.groupby('neighbourhood').sum()
    df_top_avg= df_top_exp['average_house_value'].sort_values(ascending=False).nlargest(10)
    fig_top_avg = plt.figure()
    plot_top_avg = df_top_avg.plot.bar()
    plot_top_avg.set_xlabel("neighbourhood")
    plot_top_avg.set_ylabel("average_house_value")
    plot_top_avg.set_title("The top 10 expensive neighbourhoods")
    plt.close(fig_top_exp)
    return pn.pane.Matplotlib(fig_top_avg, tight=True)
    
def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [45]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
# Create tabs
title = pn.pane.Markdown(
"""
Hello World 2020

""",
    width=900,
)
welcome = pn.pane.Markdown(
"""
This is a dashboard of Rico.....

"""
)
#layouts fro the dashboard
tabs = pn.Tabs(
        #First Tab
        (
            "Neighbourhood Map", 
            pn.Column
            (
                pn.Row
                (
                    neighbourhood_map()
                ),
                pn.Row
                (
                    neighbourhood_map()
                )
            )
        )
        )
     #second Tab
        
        (
            "average_house_value", 
            pn.Column
            (
                pn.Row
                (
                    avg_house_value()
                ),
                pn.Row
                (
                    avg_house_value()
                )
            )
        
        )
        ) 
        #3rd Tab
        
        (
            "Average house values by neighbourhood", 
            pn.Column
            (
                pn.Row
                (
                    avg_house_value()
                ),
                pn.Row
                (
                    avg_house_value()
                )
            )
        
        )
        )
        
        #4th Tab
        (
            "Number of dwelling types per year", 
            pn.Column
            (
                pn.Row
                (
                    number_of_dwelling_types_units_per_year()
                ),
                pn.Row
                (
                    number_of_dwelling_types_units_per_year()
                )
            )
        )
        )
# tabs = pn.Tabs(
#         ("neighbourhood_map", pn.Row(average_house_value())),
#         (" average_house_value", pn.Row((neighbourhood))),
#         ("average_house_value_snapshot", pn.Row((year))),
#         ("df_data_frame", pn.Row((year))),
#         ("df_top_exp", pn.Row((neighbourhood))),
#         ("Avg. dewlling", pn.Row(number_of_dwelling_types_units_per_year())),
#         ("Avg. dewlling_2", pn.Column(number_of_dwelling_types_units_per_year())),
#         ("df_num_dwelling", pn.Row(Mean_number_of_dwelling_types))),


db =  pn.Column(pn.Row(title),tabs,width=1000)
db.servable()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 85)

## Serve the Panel Dashboard

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
average_house_value()

In [ ]:
average_value_by_neighbourhood()

In [ ]:
number_dwelling_types()

In [ ]:
average_house_value_snapshot()

In [ ]:
top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()